In [1]:
import os.path
import json
from datetime import datetime, timezone

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# If modifying these scopes, delete the file token.json.
# This defines that we only want to READ calendar data.
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

In [5]:
creds = None
# The file token.json stores the user's access and refresh tokens.
# It is created automatically when the authorization flow completes for the first time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        # CRITICAL: This line looks for your credentials.json file
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

print("Authentication successful. Ready to make API calls.")
# CRITICAL: Also add token.json to your .gitignore file!

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=100569986535-u7jbn5tmg7j8vqmme5n7jnehqu3rpc38.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A41361%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=mbVaQOAqZiZL4ufyL0a1BLvgaQxfpI&access_type=offline
Authentication successful. Ready to make API calls.


In [6]:
try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.now(timezone.utc).isoformat()  # 'Z' indicates UTC time
    print("\nGetting the upcoming 10 events")
    
    events_result = (
        service.events()
        .list(
            calendarId="primary", # 'primary' is the main calendar for the logged-in user
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
        print("No upcoming events found.")
    else:
        # Print the start and name of the next 10 events
        for event in events:
            start = event["start"].get("dateTime", event["start"].get("date"))
            print(start, event["summary"])

except Exception as e:
    print(f"An error occurred: {e}")


Getting the upcoming 10 events
2025-11-10T12:10:00-05:00 CSC384H1 LEC0101 test
2025-11-11T11:00:00-05:00 MAT309H1 LEC0101
2025-11-11T15:00:00-05:00 CSC317H1 LEC0201
2025-11-12 Triangle Mastercard eStatement Payment Due Date
2025-11-12T12:00:00-05:00 CSC384H1 LEC0101
2025-11-12T15:00:00-05:00 RLG106H1 LEC0101
2025-11-13T11:00:00-05:00 MAT309H1 LEC0101
2025-11-13T15:00:00-05:00 ARIA 2025
2025-11-13T15:00:00-05:00 CSC317H1 TUT0201
2025-11-14T10:45:00-05:00 Rop meeting


In [7]:
if events:
    mock_data_dir = Path('../mock_data')
    mock_data_dir.mkdir(exist_ok=True)
    
    file_path = mock_data_dir / 'mock_calendar_events.json'
    
    with open(file_path, 'w') as f:
        json.dump(events, f, indent=4)
        
    print(f"\n✅ Successfully saved calendar events to '{file_path}'.")
else:
    print("\nNo events to save.")

NameError: name 'Path' is not defined